# Partie 4 : Faites des tests avec deux autres couches de global pooling de Pytorch Geometric.

**Travail à rendre :  (Applications pratiques des GNN) :**



*   Important : Cette partie doit être rendue et sera évaluée.
*   Deadline : Dimanche 26 janvier 2025 à 23h55.

Dans cette partie du TP, vous êtes invités à identifier un cas concret d'application unique des réseaux de neurones de graphes (GNNs) et à démontrer comment les GNNs, en combinaison avec PyG, peuvent être utilisés pour résoudre ce problème. Le cas choisi doit être basé sur une problématique réelle et pertinente. Nous encourageons l'exploration de divers domaines (ex. : biologie, finance, réseaux sociaux), en vous inspirant des exemples illustrés dans l'enoncé de ce TP.

Votre Notebook doit inclure une explication détaillée et accessible, pas uniquement du code. Veillez à :

*  Inclure une introduction claire du cas d'utilisation
*  Proposer des visualisations et des résultats interprétables
*   Fournir des explications tout au long du notebook


# Facebook

## 1. Présentation du sujet

La prédiction de liens est un problème clé en analyse de réseaux sociaux : étant donné la structure actuelle d'un réseau (par exemple Facebook, LinkedIn, etc.) et les attributs de ses nœuds, on veut estimer la probabilité qu'un lien (une amitié, un suivi, …) apparaisse entre deux nœuds qui ne sont pas encore connectés. C'est la base d'un moteur de recommandation d'amis ou de contacts.

Dans ce mini-projet, nous allons :

- Représenter chaque utilisateur comme un nœud du graphe.
- Une arête (edge) indique une amitié existante entre deux utilisateurs.
- Nous allons créer des « échantillons » positifs (des paires i,j qui ont déjà une arête) et négatifs (des paires i,j qui n'ont pas d'arête) pour entraîner un GNN à distinguer « existe un lien / n'existe pas de lien ».

Par la suite, on pourra (dans un cas d'usage réel) recommander des liens à haute probabilité d'existence aux utilisateurs.

## 2. Installation et imports

In [1]:
%pip install torch_scatter torch_sparse torch_cluster torch_spline_conv \
    -f https://data.pyg.org/whl/torch-2.5.1+cu118.html
%pip install git+https://github.com/pyg-team/pytorch_geometric.git

import os
import torch
import numpy as np
import urllib

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, SAGEConv
import torch.nn.functional as F
import random

# Pour la partie classification de graphes
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool

# Pour évaluation link prediction
from sklearn.metrics import roc_auc_score

# Utilisons le GPU s'il est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)


Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.9/950.9 kB 21.1 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-g7bnufmk
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-g7bnufmk
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 9bffcd470057f2f865aedb87815267c243806e13
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-py3-none-any.whl size=1172888 sha256=649b509c00020425fcc0fed7b93174b887c5

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:87: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:98: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:114: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:125: UserWarning: An issue occurred wh

Device: cpu


## 3. Téléchargement et préparation du sous-ensemble Facebook

Dans cette première partie, nous illustrons un cas d’usage réel d’un GNN : la prédiction de liens. L’idée est de recommander des amis potentiels dans un réseau social. Voici le pipeline :

1. Charger un sous-ensemble Facebook (liste d’arêtes “qui est ami avec qui”).
2. Construire un graphe PyG : Data(x, edge_index).
3. Séparer les arêtes en train/val/test, plus negative sampling.
4. Entraîner un GNN (GCNConv ou SAGEConv) pour obtenir des embeddings de nœuds.
5. Prédire l’existence d’une arête via un dot product des embeddings.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Ex.: un sous-réseau "0.edges" de l'archive facebook.tar.gz
EDGE_FILE = '/content/drive/My Drive/facebook/0.edges'

edges = []
unique_nodes = set()
with open(EDGE_FILE, 'r') as f:
    for line in f:
        src, dst = line.strip().split()
        src = int(src)
        dst = int(dst)
        # Exclure self-loops éventuels
        if src != dst:
            edges.append((src, dst))
            edges.append((dst, src))
        unique_nodes.update([src, dst])

unique_nodes = sorted(list(unique_nodes))
node2idx = {nid: i for i, nid in enumerate(unique_nodes)}

edge_index_list = []
for (s,d) in edges:
    edge_index_list.append([node2idx[s], node2idx[d]])
edge_index = torch.tensor(edge_index_list, dtype=torch.long).t()  # shape [2, E]

num_nodes = len(unique_nodes)
print(f"Nombre de nœuds : {num_nodes}")
print(f"Nombre d'arêtes orientées : {edge_index.size(1)}")

# Pour simplifier, on se donne un vecteur x = one-hot(num_nodes)
x = torch.eye(num_nodes)  # ou torch.ones((num_nodes,1))...

data_facebook = Data(x=x, edge_index=edge_index).to(device)


Nombre de nœuds : 333
Nombre d'arêtes orientées : 10076


## 4. Création des paires positives / négatives pour la prédiction de liens

On crée une liste “unique” d'arêtes (non orientées).
Puis on la sépare en :

  - 80 % pour train,
  - 10 % pour val,
  - 10 % pour test.

Par la suite, pour entraîner un modèle de link prediction, on a besoin :

- Paires positives : les arêtes existantes dans le graphe (en train).
- Paires négatives : échantillons de nœuds non connectés.

On crée autant de paires négatives (no link) que de paires positives.

In [5]:
# Extraire la liste d'arêtes "unique" (s<d)
unique_edges = []
visited = set()
E = edge_index.size(1)
for i in range(0, E, 2):
    s = edge_index[0, i].item()
    d = edge_index[1, i].item()
    if s > d:
        s, d = d, s
    if (s,d) not in visited:
        visited.add((s,d))
        unique_edges.append([s,d])

unique_edges = np.array(unique_edges)
np.random.shuffle(unique_edges)

n_train = int(0.8 * len(unique_edges))
n_val   = int(0.1 * len(unique_edges))

train_pos = unique_edges[:n_train]
val_pos   = unique_edges[n_train : n_train + n_val]
test_pos  = unique_edges[n_train + n_val :]

def negative_sampling(num_samples, N, forbidden_set):
    """Échantillonne des paires (s,d) non connectées dans [0..N-1]x[0..N-1]."""
    neg = []
    cpt = 0
    while cpt < num_samples:
        s = np.random.randint(0,N)
        d = np.random.randint(0,N)
        if s==d:
            continue
        if s>d:
            s,d = d,s
        if (s,d) in forbidden_set:
            continue
        neg.append([s,d])
        # On marque cette paire comme "prise" pour ne pas la ré-échantillonner
        forbidden_set.add((s,d))
        cpt += 1
    return np.array(neg)

# On crée un set “positif” existant
pos_set = set((s,d) for (s,d) in unique_edges)
train_neg = negative_sampling(len(train_pos), num_nodes, pos_set)
val_neg   = negative_sampling(len(val_pos),   num_nodes, pos_set)
test_neg  = negative_sampling(len(test_pos),  num_nodes, pos_set)

print(f"Train pos : {len(train_pos)}, Train neg : {len(train_neg)}")
print(f"Val   pos : {len(val_pos)},  Val   neg : {len(val_neg)}")
print(f"Test  pos : {len(test_pos)}, Test  neg : {len(test_neg)}")


Train pos : 2015, Train neg : 2015
Val   pos : 251,  Val   neg : 251
Test  pos : 253, Test  neg : 253


Pour être rigoureux en link prediction, on peut retirer les arêtes val/test du edge_index avant d’entraîner, afin de ne pas “révéler” ces connexions. Ci-dessous, on crée un edge_index_train qui ne contient pas les arêtes de validation/test.

In [6]:
# Supprimer val_pos et test_pos du graphe d'entraînement
train_edges_set = set()
for (s,d) in train_pos:
    train_edges_set.add((s,d))
    train_edges_set.add((d,s))

# On reconstruit un edge_index réduit
train_edge_list = []
E = edge_index.size(1)
for i in range(0, E):
    s = edge_index[0, i].item()
    d = edge_index[1, i].item()
    if (s,d) in train_edges_set:
        train_edge_list.append([s,d])

edge_index_train = torch.tensor(train_edge_list, dtype=torch.long).t().to(device)
print("Nbre d'arêtes dans edge_index_train :", edge_index_train.size(1))

# On crée un Data pour l'entraînement
data_train = Data(x=data_facebook.x, edge_index=edge_index_train).to(device)


Nbre d'arêtes dans edge_index_train : 8060


## 5. Définition du modèle GNN pour Link Prediction

On crée :

- Un encodeur GNN (GNNEncoder) basé sur SAGEConv ou GCNConv,
- Un prédicteur LinkPredictor (ici un simple dot product),
- Une classe Net pour assembler le tout et produire les logits sur des paires de nœuds.

In [7]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, num_layers=2, model_type='SAGE'):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.model_type = model_type

        if model_type == 'SAGE':
            self.convs.append(SAGEConv(in_channels, hidden_dim))
            for _ in range(num_layers-1):
                self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        elif model_type == 'GCN':
            self.convs.append(GCNConv(in_channels, hidden_dim))
            for _ in range(num_layers-1):
                self.convs.append(GCNConv(hidden_dim, hidden_dim))
        else:
            raise ValueError("Unknown model_type")

        self.dropout = 0.5

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        # Dernière couche sans relu
        x = self.convs[-1](x, edge_index)
        return x


In [9]:
class LinkPredictor(torch.nn.Module):
    """Ici, on ne fait qu'un dot product entre les embeddings, sans MLP."""
    def __init__(self, in_channels):
        super().__init__()
        # Pour un simple dot product, pas besoin de plus de paramètres.
        pass

    def forward(self, x_i, x_j):
        return (x_i * x_j).sum(dim=-1)  # logit via dot product

In [10]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, model_type='SAGE'):
        super().__init__()
        self.encoder = GNNEncoder(in_channels, hidden_dim, num_layers=2, model_type=model_type)
        self.predictor = LinkPredictor(hidden_dim)

    def forward(self, data):
        return self.encoder(data.x, data.edge_index)

    def get_link_logits(self, node_emb, pairs):
        x_i = node_emb[pairs[:,0]]
        x_j = node_emb[pairs[:,1]]
        return self.predictor(x_i, x_j)

## 6. Entraînement et évaluation

Nous allons :

1. Entraîner sur train_pos vs train_neg.
2. Évaluer sur val_pos vs val_neg.
3. Sélectionner le meilleur modèle sur la métrique ROC-AUC.
4. Tester sur test_pos vs test_neg.

In [11]:
def train_one_epoch(model, data_train, optimizer, train_pos, train_neg, batch_size=1024):
    """Apprend à distinguer train_pos vs. train_neg en BCE."""
    model.train()
    pos_label = np.ones(len(train_pos), dtype=np.float32)
    neg_label = np.zeros(len(train_neg), dtype=np.float32)

    all_pairs = np.concatenate([train_pos, train_neg], axis=0)
    all_labels = np.concatenate([pos_label, neg_label], axis=0)

    # On mélange
    idx = np.arange(len(all_pairs))
    np.random.shuffle(idx)
    all_pairs = torch.from_numpy(all_pairs[idx]).long().to(device)
    all_labels = torch.from_numpy(all_labels[idx]).float().to(device)

    total_loss = 0.0
    for start in range(0, len(all_pairs), batch_size):
        end = start + batch_size
        batch_pairs = all_pairs[start:end]
        batch_labels = all_labels[start:end]

        optimizer.zero_grad()
        node_emb = model(data_train)  # embeddings sur le graphe d'entraînement
        logits = model.get_link_logits(node_emb, batch_pairs)
        loss = F.binary_cross_entropy_with_logits(logits, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_pairs.size(0)
    return total_loss / len(all_pairs)


@torch.no_grad()
def eval_model(model, data_full, pos_edges, neg_edges, batch_size=1024):
    """
    Calcule l'AUC sur paires pos_edges vs. neg_edges.
    data_full est le graphe complet (éventuellement on n'utilise que x,
    mais on n'a pas besoin de ses arêtes sauf si on veut un autre embedding).
    """
    model.eval()
    node_emb = model(data_full)  # On peut extraire l'embedding sur tout le graphe

    # On va concaténer pos et neg, calculer logits, comparer aux labels.
    pairs = np.concatenate([pos_edges, neg_edges], axis=0)
    labels = np.concatenate([np.ones(len(pos_edges)), np.zeros(len(neg_edges))], axis=0)

    pairs = torch.from_numpy(pairs).long().to(device)
    logits = model.get_link_logits(node_emb, pairs).cpu().numpy()
    # Sigmoid pour obtenir un score [0..1]
    scores = 1/(1+np.exp(-logits))

    auc_val = roc_auc_score(labels, scores)
    return auc_val


In [12]:
model = Net(in_channels=data_facebook.x.size(1), hidden_dim=64, model_type='SAGE').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

best_val_auc = 0
best_state = None
epochs = 30

for epoch in range(1, epochs+1):
    loss = train_one_epoch(model, data_train, optimizer, train_pos, train_neg, batch_size=512)
    val_auc = eval_model(model, data_facebook, val_pos, val_neg)

    if val_auc > best_val_auc:
        best_val_auc = val_auc
        best_state = {k: v.cpu() for k,v in model.state_dict().items()}

    if epoch % 5 == 0:
        print(f"[Epoch {epoch:02d}] loss={loss:.4f} val_auc={val_auc:.4f}")

# Charger le meilleur
model.load_state_dict({k: v.to(device) for k,v in best_state.items()})
test_auc = eval_model(model, data_facebook, test_pos, test_neg)
print(f"Meilleur modèle : val_auc={best_val_auc:.4f} => test_auc={test_auc:.4f}")

[Epoch 05] loss=0.4606 val_auc=0.9245
[Epoch 10] loss=0.3884 val_auc=0.9297
[Epoch 15] loss=0.3459 val_auc=0.9195
[Epoch 20] loss=0.3014 val_auc=0.9050
[Epoch 25] loss=0.2717 val_auc=0.8986
[Epoch 30] loss=0.2546 val_auc=0.8840
Meilleur modèle : val_auc=0.9338 => test_auc=0.8825


On obtient un score AUC mesurant la capacité du modèle à distinguer les paires connectées vs. non connectées (plus l'AUC est proche de 1, meilleure est la performance).

# Les enzymes

Dans cette deuxieme partie nous allons répondre à la Partie 4 du TP : « tester deux couches de global pooling », nous illustrons ici un autre problème : la classification de graphes.

Pour se faire, nous allons utiliser un dataset TUDataset (par exemple ENZYMES), l'idée est :

- Chaque exemple est un petit graphe.
- On veut prédire la classe ou le label associé à ce graphe (ex. type d'enzyme).
- Pour agréger l’information des nœuds en un seul vecteur de graphe, on compare :
        global_mean_pool vs. global_add_pool.

## 1. Chargement du dataset

Ici, on prend le dataset ENZYMES, qui contient 600 graphes, chacun étiqueté en 6 classes.

In [13]:
from torch_geometric.datasets import TUDataset

# root = './mutag_data'
# name = 'MUTAG'

root = './enzymes_data'
name = 'ENZYMES'
dataset_enz = TUDataset(root=root, name=name)

print(dataset_enz)
print(f"Nombre de graphes = {len(dataset_enz)}")
print(f"Nombre de classes = {dataset_enz.num_classes}")
print(f"Dimension des features nœuds = {dataset_enz.num_features}")

# Pour la démo, on fait un split train/val/test (60/20/20)
nb_graphs = len(dataset_enz)
indices = list(range(nb_graphs))
random.shuffle(indices)

train_ratio = 0.6
val_ratio = 0.2
ntrain = int(train_ratio*nb_graphs)
nval   = int(val_ratio*nb_graphs)

train_idx = indices[:ntrain]
val_idx   = indices[ntrain : ntrain+nval]
test_idx  = indices[ntrain+nval:]

train_dataset = dataset_enz[train_idx]
val_dataset   = dataset_enz[val_idx]
test_dataset  = dataset_enz[test_idx]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


Processing...


ENZYMES(600)
Nombre de graphes = 600
Nombre de classes = 6
Dimension des features nœuds = 3
Train: 360, Val: 120, Test: 120


Done!


## 2. Définition d'un GNN pour classification de graphes

Nous réutilisons un GCN + un pooling global + un linear final.
0n rend le pooling paramétrable (pooling_type), et on va tester mean vs. add.

In [14]:
class GCN_Graph(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers, dropout, pooling_type='mean'):
        super().__init__()

        # Liste de couches GCNConv
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_dim, hidden_dim))
        for _ in range(num_layers-2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.convs.append(GCNConv(hidden_dim, out_dim))  # sortie = out_dim

        # Liste de BatchNorm (pour les couches intermédiaires)
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(hidden_dim) for _ in range(num_layers-1)
        ])

        self.dropout = dropout

        # Pooling global : add ou mean
        if pooling_type=='add':
            self.pool = global_add_pool
        else:
            # défaut = mean
            self.pool = global_mean_pool

    def forward(self, batch_data):
        x, edge_index, batch = batch_data.x, batch_data.edge_index, batch_data.batch

        for i in range(len(self.convs)-1):
            x = self.convs[i](x, edge_index)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        # dernière couche sans BN ni relu, par ex.
        x = self.convs[-1](x, edge_index)

        # on obtient des features par nœud => pooling global
        graph_emb = self.pool(x, batch)
        return graph_emb

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()


Nous pouvons soit incorporer la projection finale dans la dernière couche du GCN (c'est déjà out_dim = num_classes), soit ajouter un Linear. Ici, comme nous avons mis out_dim = num_classes, il suffit de faire un LogSoftmax sur le vecteur agrégé.

In [15]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_examples = 0

    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        emb = model(batch)  # shape [batch_size, num_classes]
        # Le label est batch.y (shape [batch_size]) => cross-entropy
        loss = criterion(emb, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
        total_examples += batch.num_graphs
    return total_loss / total_examples

@torch.no_grad()
def eval_model_classif(model, loader):
    model.eval()
    correct = 0
    total = 0
    for batch in loader:
        batch = batch.to(device)
        out = model(batch)  # [batch_size, num_classes]
        pred = out.argmax(dim=-1)  # classes
        correct += (pred == batch.y).sum().item()
        total += batch.num_graphs
    return correct / total


## 3. Comparaison pooling=mean vs. pooling=add

Entraînons deux fois le même GNN, juste en changeant le type de pooling.

In [16]:
def run_experiment(pooling_type='mean'):
    model = GCN_Graph(
        in_dim=dataset_enz.num_features,
        hidden_dim=64,
        out_dim=dataset_enz.num_classes,
        num_layers=3,
        dropout=0.5,
        pooling_type=pooling_type
    ).to(device)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    best_val_acc = 0
    best_state = None
    nb_epochs = 30

    for epoch in range(1, nb_epochs+1):
        loss = train_epoch(model, train_loader, optimizer, criterion)
        train_acc = eval_model_classif(model, train_loader)
        val_acc   = eval_model_classif(model, val_loader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {k: v.cpu() for k,v in model.state_dict().items()}

        if epoch%10==0:
            print(f"[{pooling_type}] Epoch {epoch:02d} | loss={loss:.4f} | "
                  f"train_acc={train_acc:.3f} | val_acc={val_acc:.3f}")

    # On recharge le meilleur
    model.load_state_dict({k: v.to(device) for k,v in best_state.items()})
    test_acc = eval_model_classif(model, test_loader)
    print(f"=> Pooling={pooling_type}, Best val_acc={best_val_acc:.3f}, Test acc={test_acc:.3f}")
    return test_acc

print("=== Test 1 : global_mean_pool ===")
test_acc_mean = run_experiment(pooling_type='mean')

print("\n=== Test 2 : global_add_pool ===")
test_acc_add = run_experiment(pooling_type='add')


=== Test 1 : global_mean_pool ===
[mean] Epoch 10 | loss=1.7139 | train_acc=0.331 | val_acc=0.242
[mean] Epoch 20 | loss=1.7050 | train_acc=0.325 | val_acc=0.300
[mean] Epoch 30 | loss=1.6815 | train_acc=0.250 | val_acc=0.233
=> Pooling=mean, Best val_acc=0.300, Test acc=0.250

=== Test 2 : global_add_pool ===
[add] Epoch 10 | loss=4.8810 | train_acc=0.236 | val_acc=0.225
[add] Epoch 20 | loss=2.5756 | train_acc=0.289 | val_acc=0.208
[add] Epoch 30 | loss=2.1003 | train_acc=0.236 | val_acc=0.175
=> Pooling=add, Best val_acc=0.283, Test acc=0.208


Les résultats montrent que l'utilisation de global_mean_pool offre une meilleure précision sur le test que global_add_pool pour la classification de graphes, ce qui suggère que l'agrégation moyenne peut capturer les caractéristiques des graphes de manière plus efficace dans ce contexte particulier. Toutefois, la performance peut varier en fonction du dataset et de l'architecture utilisée.

# Conclusion

**Link Prediction (Partie : Facebook)**
- Nous avons illustré un cas d'usage concret des GNN : la recommandation d'amis.
- Le modèle GNN encode chaque utilisateur via SAGEConv/GCNConv, puis un dot product estime la probabilité de lien.
- Nous avons évalué la qualité de prédiction via l'AUC sur paires positives/négatives.

Classification de graphes (Partie: ENZYMES)
- Nous avons construit un GCN pour classifier des graphes ENZYMES, avec un pooling global.
- Nous avons comparé global_mean_pool et global_add_pool.
- Les performances peuvent différer selon la nature du dataset et l'architecture.